Imports

In [ ]:
import numpy as np
import nibabel as nib                                                     
import matplotlib.pyplot as plt
from skimage.util import montage 
from skimage.transform import rotate
import seaborn as sns
import tensorflow as tf
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import cv2
import tarfile
import pandas as pd
import numpy as np
import nibabel as nib
import glob
import matplotlib.pyplot as plt
from tifffile import imsave
from zipfile import ZipFile
import nibabel as nib
import os
import random
from tensorflow import keras
import tensorflow as tf
import datetime
import tarfile

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.models import Model, load_model
from keras.layers import Input ,BatchNormalization , Activation 
from keras.layers.convolutional import Conv2D, UpSampling2D
from keras.layers.pooling import MaxPooling2D
from keras.layers.merge import concatenate
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras import optimizers 
from sklearn.model_selection import train_test_split
import os
import nibabel as nib
import cv2 as cv
import matplotlib.pyplot as plt
from keras import backend as K
import glob
import skimage.io as io
import skimage.color as color
import random as r
import math


Create Training Data

In [ ]:
data = [i for i in os.listdir("brain_images/") if "BraTS2021_" in i][:200]
from tqdm import tqdm
X = np.zeros((len(data),240, 240,155, 4),dtype='float32')
y = np.zeros((len(data),240, 240, 155,1),dtype='float32')
c = 0
for file in tqdm(data):
  path_file = "brain_images/"+file+"/"
  flair = nib.load(path_file+file+"_flair.nii.gz").get_fdata()
  t1 = nib.load(path_file+file+"_t1.nii.gz").get_fdata()
  t1ce = nib.load(path_file+file+"_t1ce.nii.gz").get_fdata()
  t2 = nib.load(path_file+file+"_t2.nii.gz").get_fdata()
  seg = nib.load(path_file+file+"_seg.nii.gz").get_fdata()#
  X[c,:,:,:,0] = flair
  X[c,:,:,:,1] = t1
  X[c,:,:,:,2] = t1ce
  X[c,:,:,:,3] = t2
  y[c,:,:,:,0] = seg
  c+=1

100%|██████████| 200/200 [02:35<00:00,  1.29it/s]


Save Training Data

In [ ]:
np.save("drive/MyDrive/BraTS/X",X)
np.save("drive/MyDrive/BraTS/y",y)


Load Training Data

In [ ]:
X = np.load("drive/MyDrive/BraTS/X.npy",mmap_mode="r")[:100]
y = np.load("drive/MyDrive/BraTS/y.npy",mmap_mode="r")[:100]
maximum = np.amax(X)

Institution Sharding

In [ ]:
institutions = [0,5,10,15,20,30,40,50,70,100]

ins_arrays_X = [X[institutions[i]:institutions[i+1]] for i in range(9)]
ins_arrays_y = [y[institutions[i]:institutions[i+1]] for i in range(9)]
for i in range(9):
  print(ins_arrays_X[i].shape, ins_arrays_y[i].shape)

(5, 240, 240, 155, 4) (5, 240, 240, 155, 1)
(5, 240, 240, 155, 4) (5, 240, 240, 155, 1)
(5, 240, 240, 155, 4) (5, 240, 240, 155, 1)
(5, 240, 240, 155, 4) (5, 240, 240, 155, 1)
(10, 240, 240, 155, 4) (10, 240, 240, 155, 1)
(10, 240, 240, 155, 4) (10, 240, 240, 155, 1)
(10, 240, 240, 155, 4) (10, 240, 240, 155, 1)
(20, 240, 240, 155, 4) (20, 240, 240, 155, 1)
(30, 240, 240, 155, 4) (30, 240, 240, 155, 1)


In [ ]:
institutions_t = [0,4,8,12,16,24,32,40,56,80]#[0,5,10,20,30,45,65,90,120,155,200]
weights = [(institutions_t[i+1]-institutions_t[i])/80 for i in range(9)]
print(sum(weights),weights)

1.0 [0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.1, 0.2, 0.3]


In [ ]:
k=0
ins_valid_split = [[k*(ins_arrays_X[i].shape[0]//5)+split for split in range(ins_arrays_X[i].shape[0]//5)] for i in range(9)]
ins_train_split = [[u for u in range(ins_arrays_X[i].shape[0]) if u not in ins_valid_split[i]] for i in range(9)]
print(ins_valid_split)
print(ins_train_split)

[[0], [0], [0], [0], [0, 1], [0, 1], [0, 1], [0, 1, 2, 3], [0, 1, 2, 3, 4, 5]]
[[1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [1, 2, 3, 4], [2, 3, 4, 5, 6, 7, 8, 9], [2, 3, 4, 5, 6, 7, 8, 9], [2, 3, 4, 5, 6, 7, 8, 9], [4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19], [6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29]]


In [ ]:
X_valid = []
y_valid = []
X_train = []
y_train = []
for ins in range(9):
  for split in ins_valid_split[ins]:
    for j in range(155):
      X_valid.append(ins_arrays_X[ins][split,:,:,j])
      y_valid.append(ins_arrays_y[ins][split,:,:,j])
  for split in ins_train_split[ins]:
    for j in range(155):
      X_train.append(ins_arrays_X[ins][split,:,:,j])
      y_train.append(ins_arrays_y[ins][split,:,:,j])
X_valid = np.array(X_valid)
y_valid = np.array(y_valid)
X_train = np.array(X_train)
y_train = np.array(y_train)

print(X_train.shape,y_train.shape,X_valid.shape,y_valid.shape)


(12400, 240, 240, 4) (12400, 240, 240, 1) (3100, 240, 240, 4) (3100, 240, 240, 1)


Create UNet

In [ ]:
initializer=tf.random_normal_initializer(0., 0.02)

tf.keras.mixed_precision.set_global_policy('float32')
def Convolution(input_tensor,filters,dropout=False):
    
    x = Conv2D(filters=filters,kernel_size=(4, 4),padding = 'same',strides=(2, 2),kernel_initializer=initializer,use_bias=False)(input_tensor)
    x = BatchNormalization()(x)
    if dropout:
      x=keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.LeakyReLU()(x) 
    return x
from keras.layers import Conv2DTranspose
def ConvolutionTransposed(input_tensor,filters,dropout=False):
    
    x = Conv2DTranspose(filters=filters,kernel_size=(4, 4),padding = 'same',strides=(2, 2),kernel_initializer=initializer,use_bias=False)(input_tensor)
    x = BatchNormalization()(x)
    if dropout:
      x=keras.layers.Dropout(0.5)(x)
    x = tf.keras.layers.ReLU()(x) 
    return x
def model(input_shape):
    
    inputs = Input((input_shape))
    input=tf.keras.layers.ZeroPadding2D(padding=((8,8),(8,8)))(inputs)
    print(input.shape)
    conv_1 = Convolution(input,32)
    
    conv_2 = Convolution(conv_1,64)
    
    conv_3 = Convolution(conv_2,128,dropout=True)
    
    conv_4 = Convolution(conv_3,256,dropout=True)
    
    conv_5 = Convolution(conv_4,512,dropout=True)#

    conv_6 = Convolution(conv_5,512,dropout=True)

    conv_7 = Convolution(conv_6,512,dropout=True)

    conv_8 = Convolution(conv_7,1024,dropout=True)

    trans_7=ConvolutionTransposed(conv_8,512,dropout=True)

    trans_6=ConvolutionTransposed(keras.layers.Concatenate()([trans_7,conv_7]),512,dropout=True)

    trans_5=ConvolutionTransposed(keras.layers.Concatenate()([trans_6,conv_6]),512,dropout=True)

    trans_4=ConvolutionTransposed(keras.layers.Concatenate()([trans_5,conv_5]),256,dropout=True)
    trans_3=ConvolutionTransposed(keras.layers.Concatenate()([trans_4,conv_4]),128,dropout=True)

    trans_2=ConvolutionTransposed(keras.layers.Concatenate()([trans_3,conv_3]),64,dropout=True)

    trans_1=ConvolutionTransposed(keras.layers.Concatenate()([trans_2,conv_2]),32)

    outputs = Conv2DTranspose(1,4, (2, 2), activation='sigmoid',padding="same",dtype='float32')(keras.layers.Concatenate()([trans_1,conv_1]))[:,8:-8,8:-8,:]
    print(outputs.shape)
    model = Model(inputs=[inputs], outputs=[outputs]) 
    
    return model
#model = model(input_shape = (240,240,4))
def dice_coef(y_true, y_pred, smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

def dice_log(y_true,y_pred,smooth=1.0):
    y_true_f = K.flatten(y_true)
    y_pred_f = K.flatten(y_pred)
    intersection = K.sum(y_true_f * y_pred_f)
    return tf.math.log((K.sum(y_true_f) + K.sum(y_pred_f) + smooth))-tf.math.log((2. * intersection + smooth))
#model = model(input_shape = (240,240,4))
#model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=dice_log)

Centralised Model Training

In [ ]:
EPOCHS = 300
BATCH_SIZE=64
scores_cl_e2 = []
model1 = model(input_shape = (240,240,4))
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=dice_log)
import gc
from tqdm import tqdm
with tf.device("GPU:0"):
  for epoch in range(EPOCHS):
    print("Epoch "+str(epoch+1))
    for iteration in tqdm(range(len(X_train)//BATCH_SIZE)):
      batch_indices = [random.randrange(0,len(X_train)) for i in range(BATCH_SIZE)]
      model1.train_on_batch((X_train[batch_indices])/maximum,(y_train[batch_indices])/4)
    result = np.empty((3100,240,240,1),dtype='float32')
    for i in tqdm(range(20)):
      result[i*155:(i+1)*155]=model1.predict(X_valid[i*155:(i+1)*155]/maximum)
      gc.collect()
    dice_score = dice_coef(y_valid/4,result)
    scores_cl_e2.append(dice_score)
    print(dice_score)
    gc.collect()
np.save("drive/MyDrive/BraTS/scores_cl_e2",np.array(scores_cl_e2))
  


(None, 256, 256, 4)
(None, 240, 240, 1)
Epoch 1


100%|██████████| 20/20 [00:19<00:00,  1.04it/s]


tf.Tensor(0.03483042, shape=(), dtype=float32)
Epoch 2


100%|██████████| 20/20 [00:15<00:00,  1.25it/s]


tf.Tensor(0.4780043, shape=(), dtype=float32)
Epoch 3


...

tf.Tensor(0.6284199, shape=(), dtype=float32)
Epoch 299


100%|██████████| 20/20 [00:15<00:00,  1.26it/s]


tf.Tensor(0.6280555, shape=(), dtype=float32)
Epoch 300


100%|██████████| 20/20 [00:15<00:00,  1.26it/s]


tf.Tensor(0.62672794, shape=(), dtype=float32)



CIIL



In [ ]:
EPOCHS = 34
BATCH_SIZE=64
scores_ciil_e2 = []
model1 = model(input_shape = (240,240,4))
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=dice_log)
import gc
from tqdm import tqdm
with tf.device("GPU:0"):
  for epoch in range(EPOCHS):
    print("Epoch "+str(epoch+1))
    for ins in range(9):
      length = len(X_train[155*institutions_t[ins]:155*institutions_t[ins+1]])
      for iteration in tqdm(range(length//BATCH_SIZE)):
        batch_indices = [random.randrange(0,length) for i in range(BATCH_SIZE)]
        model1.train_on_batch((X_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/maximum,(y_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/4)
      result = np.empty((3100,240,240,1),dtype='float32')
      for i in tqdm(range(20)):
        result[i*155:(i+1)*155]=model1.predict(X_valid[i*155:(i+1)*155]/maximum)
      dice_score = dice_coef(y_valid/4,result)
      scores_ciil_e2.append(dice_score)
      print(dice_score)
      gc.collect()
np.save("drive/MyDrive/BraTS/scores_ciil_e2",np.array(scores_ciil_e2))  


(None, 256, 256, 4)
(None, 240, 240, 1)
Epoch 1


100%|██████████| 20/20 [00:15<00:00,  1.31it/s]


tf.Tensor(0.012134422, shape=(), dtype=float32)


100%|██████████| 20/20 [00:12<00:00,  1.66it/s]


tf.Tensor(0.015609153, shape=(), dtype=float32)


...

tf.Tensor(0.63112104, shape=(), dtype=float32)


100%|██████████| 20/20 [00:11<00:00,  1.68it/s]


tf.Tensor(0.6347294, shape=(), dtype=float32)


100%|██████████| 20/20 [00:11<00:00,  1.69it/s]


tf.Tensor(0.62153447, shape=(), dtype=float32)


FL

In [ ]:

def get_avg_weights(array_weights,weights):
    new_weights = list()
    for weights_list_tuple in zip(*array_weights):
        new_weights.append(
            np.array([np.average(np.array(w),axis=0,weights=weights) for w in zip(*weights_list_tuple)])
        )
    return new_weights

def fl_average(models,weighted):
    weights = [model.get_weights() for model in models]
    weights = get_avg_weights(weights,weighted)
    return weights
def fl_average_givenweights(weights,weighted):
    weights = get_avg_weights(weights,weighted)
    return weights

In [ ]:
weights = [(institutions_t[i+1]-institutions_t[i])/80 for i in range(9)]
model_list=[]
for i in range(9):
  model1 = model(input_shape = (240,240,4))
  model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=dice_log)
  model_list.append(model1)
EPOCHS = 300
BATCH_SIZE=64
scores_fl_e3 = []
import gc
from tqdm import tqdm
with tf.device("GPU:0"):
  for epoch in range(EPOCHS):
    print("Epoch "+str(epoch+1))
    for ins in range(9):
      length = len(X_train[155*institutions_t[ins]:155*institutions_t[ins+1]])
      for iteration in tqdm(range(length//BATCH_SIZE)):
        batch_indices = [random.randrange(0,length) for i in range(BATCH_SIZE)]
        model_list[ins].train_on_batch((X_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/maximum,(y_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/4)
      
    model_weights = fl_average(model_list,weights)
    for ins in range(9):
      model_list[ins].set_weights(model_weights)
    result = np.empty((3100,240,240,1),dtype='float32')
    gc.collect()
    for i in tqdm(range(155)):
      result[i*20:(i+1)*20]=model_list[0].predict(X_valid[i*20:(i+1)*20]/maximum)
      #gc.collect()
    dice_score = dice_coef(y_valid/4,result)
    scores_fl_e3.append(dice_score)
    print(dice_score)
    gc.collect()
np.save("drive/MyDrive/BraTS/scores_fl_e3",np.array(scores_fl_e3))


(None, 256, 256, 4)
(None, 240, 240, 1)
(None, 256, 256, 4)
(None, 240, 240, 1)
(None, 256, 256, 4)
(None, 240, 240, 1)
(None, 256, 256, 4)
(None, 240, 240, 1)
(None, 256, 256, 4)
(None, 240, 240, 1)
(None, 256, 256, 4)
(None, 240, 240, 1)
(None, 256, 256, 4)
(None, 240, 240, 1)
(None, 256, 256, 4)
(None, 240, 240, 1)
(None, 256, 256, 4)
(None, 240, 240, 1)
Epoch 1


100%|██████████| 155/155 [00:19<00:00,  7.91it/s]


tf.Tensor(0.012036543, shape=(), dtype=float32)
Epoch 2


100%|██████████| 155/155 [00:17<00:00,  8.76it/s]


tf.Tensor(0.013692092, shape=(), dtype=float32)
Epoch 3


...

tf.Tensor(0.61933297, shape=(), dtype=float32)
Epoch 299


100%|██████████| 155/155 [00:18<00:00,  8.50it/s]


tf.Tensor(0.6147516, shape=(), dtype=float32)
Epoch 300


100%|██████████| 155/155 [00:18<00:00,  8.50it/s]


tf.Tensor(0.61181694, shape=(), dtype=float32)


EBR

In [ ]:
weights = [(institutions_t[i+1]-institutions_t[i])/80 for i in range(9)]
weight_list=[0 for i in range(9)]
model1 = model(input_shape = (240,240,4))
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=dice_log)

replay_factor = 0.3
EPOCHS = 34
BATCH_SIZE=64
scores_ebr_03_e1 = []
import gc
from tqdm import tqdm
with tf.device("GPU:0"):
  for epoch in range(EPOCHS):
    print("Epoch "+str(epoch+1))
    for ins in range(9):
      length = len(X_train[155*institutions_t[ins]:155*institutions_t[ins+1]])
      for iteration in tqdm(range(length//BATCH_SIZE)):
        batch_indices = [random.randrange(0,length) for i in range(BATCH_SIZE)]
        model1.train_on_batch((X_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/maximum,(y_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/4)
      weight_list[ins]=model1.get_weights()
      if epoch >0:
        weights = [9-ins+i for i in range(0,ins)]+[0,]+[-ins+i for i in range(ins+1,9)]
        weights_sum = sum(weights)
        weights = [weights[i]*replay_factor for i in range(9)]
        weights[ins]=(1-replay_factor)*weights_sum
        model_weights = fl_average_givenweights(weight_list,weights)
        model1.set_weights(model_weights)
        #weight_list[ins]=model_weights
      result = np.empty((3100,240,240,1),dtype='float32')
      gc.collect()
      for i in tqdm(range(100)):
        result[i*31:(i+1)*31]=model1.predict(X_valid[i*31:(i+1)*31]/maximum)
        #gc.collect()
      dice_score = dice_coef(y_valid/4,result)
      scores_ebr_03_e1.append(dice_score)
      print(dice_score)
      gc.collect()
np.save("drive/MyDrive/BraTS/scores_ebr_03_e1",np.array(scores_ebr_03_e1))

(None, 256, 256, 4)
(None, 240, 240, 1)
Epoch 1


100%|██████████| 100/100 [00:14<00:00,  7.08it/s]


tf.Tensor(0.011915241, shape=(), dtype=float32)


100%|██████████| 100/100 [00:13<00:00,  7.67it/s]


tf.Tensor(0.017454542, shape=(), dtype=float32)


...

tf.Tensor(0.61626315, shape=(), dtype=float32)


100%|██████████| 100/100 [00:13<00:00,  7.29it/s]


tf.Tensor(0.62622, shape=(), dtype=float32)


100%|██████████| 100/100 [00:13<00:00,  7.52it/s]


tf.Tensor(0.61883116, shape=(), dtype=float32)


EBR with Batch Weighting

In [ ]:
weights = [(institutions_t[i+1]-institutions_t[i])/80 for i in range(9)]
weight_list=[0 for i in range(9)]
model1 = model(input_shape = (240,240,4))
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=dice_log)

replay_factor = 0.5
EPOCHS = 34
BATCH_SIZE=64
scores_ebr_b_05_e1 = []
gc.collect()
import gc
from tqdm import tqdm
with tf.device("GPU:0"):
  for epoch in range(EPOCHS):
    print("Epoch "+str(epoch+1))
    for ins in range(9):
      length = len(X_train[155*institutions_t[ins]:155*institutions_t[ins+1]])
      for iteration in tqdm(range(length//BATCH_SIZE)):
        batch_indices = [random.randrange(0,length) for i in range(BATCH_SIZE)]
        model1.train_on_batch((X_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/maximum,(y_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/4)
      weight_list[ins]=model1.get_weights()
      if epoch >0:
        weights1 = [(9-ins+i)*weights[i] for i in range(0,ins)]+[0,]+[(-ins+i)*weights[i]  for i in range(ins+1,9)]
        weights_sum = sum(weights1)
        weights1 = [weights1[i]*replay_factor for i in range(9)]
        weights1[ins]=(1-replay_factor)*weights_sum*weights[ins]
        model_weights = fl_average_givenweights(weight_list,weights1)
        model1.set_weights(model_weights)
        #weight_list[ins]=model_weights
      result = np.empty((3100,240,240,1),dtype='float32')
      gc.collect()
      for i in tqdm(range(20)):
        result[i*155:(i+1)*155]=model1.predict(X_valid[i*155:(i+1)*155]/maximum)
        gc.collect()
      dice_score = dice_coef(y_valid/4,result)
      scores_ebr_b_05_e1.append(dice_score)
      print(dice_score)
      gc.collect()
np.save("drive/MyDrive/BraTS/scores_ebr_b_05_e1",np.array(scores_ebr_b_05_e1))

(None, 256, 256, 4)
(None, 240, 240, 1)
Epoch 1


100%|██████████| 20/20 [00:15<00:00,  1.30it/s]


tf.Tensor(0.011888243, shape=(), dtype=float32)


100%|██████████| 20/20 [00:15<00:00,  1.32it/s]


tf.Tensor(0.01222059, shape=(), dtype=float32)


...

tf.Tensor(0.617882, shape=(), dtype=float32)


100%|██████████| 20/20 [00:15<00:00,  1.30it/s]


tf.Tensor(0.61682045, shape=(), dtype=float32)


100%|██████████| 20/20 [00:15<00:00,  1.29it/s]


tf.Tensor(0.61422634, shape=(), dtype=float32)


Running Average

In [ ]:
weights = [(institutions_t[i+1]-institutions_t[i])/80 for i in range(9)]
weight_list=[0 for i in range(9)]
model1 = model(input_shape = (240,240,4))
model1.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss=dice_log)

replay_factor = 0.5
EPOCHS = 34
BATCH_SIZE=64
scores_ebr_2_e1 = []
average_running = 0
next_weights = 0
import gc
from tqdm import tqdm
with tf.device("GPU:0"):
  for epoch in range(EPOCHS):
    print("Epoch "+str(epoch+1))
    for ins in range(9):
      length = len(X_train[155*institutions_t[ins]:155*institutions_t[ins+1]])
      for iteration in tqdm(range(length//BATCH_SIZE)):
        batch_indices = [random.randrange(0,length) for i in range(BATCH_SIZE)]
        model1.train_on_batch((X_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/maximum,(y_train[155*institutions_t[ins]:155*institutions_t[ins+1]][batch_indices])/4)

      current_weights=model1.get_weights()
      if epoch == 0 and ins == 0:
        average_running=current_weights
      next_weights =  fl_average_givenweights([current_weights,average_running],[1-replay_factor,replay_factor])
      average_running = fl_average_givenweights([current_weights,average_running],[2,7])
      model1.set_weights(next_weights)

      result = np.empty((3100,240,240,1),dtype='float32')
      gc.collect()
      for i in tqdm(range(20)):
        result[i*155:(i+1)*155]=model1.predict(X_valid[i*155:(i+1)*155]/maximum)
        gc.collect()
      dice_score = dice_coef(y_valid/4,result)
      scores_ebr_2_e1.append(dice_score)
      print(dice_score)
      gc.collect()

np.save("drive/MyDrive/BraTS/scores_ebr_2_e1",np.array(scores_ebr_2_e1))